### 감정 분석
- 텍스트에 숨겨져 있는 저자의 의도나 정보를 찾아내는 모든 방법
- 데이터 수집 후 전처리 작업, 형태소 분석, 임베딩 작업 후 머신러닝, 딥러닝을 통해 분석 작업

> 네이버 영화평 
>
> 원본 데이터 소스 : https://github.com/e9t/nsmc
> 

#### Embedding을 이용한 영화평 긍정/부정 예측 이진 분류 모델 


| **비교 항목**              | **OpenAI Embedding API**                          | **딥러닝 모델로 직접 임베딩 생성**                     |
|----------------------------|---------------------------------------------------|-------------------------------------------------------|
| **1. 개발 및 구현 난이도**   | ✅ **간단하고 빠름**: 이미 학습된 모델을 호출 | 🚧 **복잡함**: 모델 설계, 학습, 튜닝 필요 |
| **2. 초기 설정 비용**       | ✅ **거의 없음**: OpenAI 클라이언트 호출로 즉시 사용 가능 | 🚧 **높음**: 데이터 준비, 모델 학습, 하드웨어 필요 |
| **3. 모델 품질**            | ✅ **최신 기술**: OpenAI가 최적화된 사전 학습 모델 제공 | 품질이 데이터와 학습 과정에 크게 의존 |
| **4. 사용자 맞춤화**        | ❌ **제한적**: 제공된 모델을 변경하거나 재학습 불가 | ✅ **유연함**: 사용자의 데이터로 모델을 재학습 가능 |
| **8. 대규모 데이터 학습**    | ❌ **불가능**: OpenAI 모델은 이미 고정된 학습 데이터 기반 | ✅ **가능**: 특정 도메인 데이터를 학습해 성능 개선 가능 |
| **9. 도메인 적합성**         | ❌ **제한적**: OpenAI 모델이 일반화된 데이터에 최적화 | ✅ **높음**: 특정 도메인 데이터로 커스터마이징 가능 |
| **10. 유지보수와 지속성**    | ✅ **OpenAI가 관리**: API 호출만 하면 됨 | 🚧 **직접 관리**: 코드, 학습 데이터, 모델 업데이트 필요 |

In [7]:
from dotenv import load_dotenv, find_dotenv
import numpy as np
from numpy import dot
import pandas as pd
from numpy.linalg import norm
from openai import OpenAI

client = OpenAI()

# .env 를 환경변수로 설정
load_dotenv(find_dotenv())

True

In [2]:
DATA_TRAIN_PATH = "https://raw.githubusercontent.com/sollab-source/sollab-source.github.io/main/data/ratings_train.txt"
DATA_TEST_PATH = "https://raw.githubusercontent.com/sollab-source/sollab-source.github.io/main/data/ratings_test.txt"

In [3]:
train_df = pd.read_csv(DATA_TRAIN_PATH,delimiter="\t")
test_df = pd.read_csv(DATA_TEST_PATH,delimiter="\t")

In [4]:
print(train_df.head(5))
print(test_df.head(5))

         id                                           document  label
0   9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1   3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2  10265843                                  너무재밓었다그래서보는것을추천한다      0
3   9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4   6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1
        id                                           document  label
0  6270596                                                굳 ㅋ      1
1  9274899                               GDNTOPCLASSINTHECLUB      0
2  8544678             뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아      0
3  6825595                   지루하지는 않은데 완전 막장임... 돈주고 보기에는....      0
4  6723715  3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??      0


In [5]:
print(train_df.shape) # (150000, 3) -> 긍 500 / 부 500 로 추출
print(test_df.shape) # (50000, 3) -> 100 / 100

(150000, 3)
(50000, 3)


In [25]:
n_train_samples = 500
n_test_samples = 100

# 긍정 부정 리뷰 추출
train_positive = train_df[train_df.label == 1]
train_negative = train_df[train_df.label == 0]
# 긍정 부정 리뷰 추출(시험 데이터)
test_positive = test_df[test_df.label == 1]
test_negative = test_df[test_df.label == 0]

# 추출된 데이터에서 무작위로 500개 추출 / 시험 데이터 100개
train_positive_sample = train_positive.sample(n=n_train_samples, random_state=42)
train_negative_sample = train_negative.sample(n=n_train_samples, random_state=42)
test_positive_sample = test_positive.sample(n=n_test_samples, random_state=42)
test_negative_sample = test_negative.sample(n=n_test_samples, random_state=42)

# 학습데이터, 시험데이터 결합
train_sample = pd.concat([train_positive_sample,train_negative_sample],ignore_index=True)
train_sample.to_csv("./data/train_sample.csv",index=False)
test_sample = pd.concat([test_positive_sample,test_negative_sample],ignore_index=True)

# 확인
print(train_sample['label'].value_counts())
print(test_sample['label'].value_counts())

label
1    500
0    500
Name: count, dtype: int64
label
1    100
0    100
Name: count, dtype: int64


In [8]:
# 임베딩 생성(사이킷런 등 라이브러리를 이용하면 직접 가능하긴함)
# 그러나 현재는 openai활용

# batch_size => 한꺼번에 ~ 만큼 가져와라
def get_embedding(text, batch_size=256):
    vectors = []
    for i in range(0,len(text),batch_size):
        batch = text[i:i+batch_size]

        response = client.embeddings.create(
        input=batch,
        model="text-embedding-3-small"
        )
        vectors.extend([d.embedding for d in response.data])
    return np.array(vectors, dtype=np.float32)

In [ ]:
# document 내용
train_text = train_sample.document.to_list()
test_text = test_sample.document.to_list()

X_train = get_embedding(train_text)
X_test = get_embedding(test_text)

# 받은 임베딩을 numpy 형태로 저장
np.save("./data/X_train.npy",X_train)
np.save("./data/X_test.npy",X_test)

In [11]:
# npy 로 저장한 임베딩 파일 불러오기
X_train_emb = np.load("./data/X_train.npy")
X_test_emb = np.load("./data/X_test.npy")

In [14]:
X_train= X_train_emb
y_train = train_sample.label.values

x_test = X_test_emb
y_test = test_sample.label.values

X_train.shape,y_train.shape,X_test.shape,y_test.shape

((1000, 1536), (1000,), (200, 1536), (200,))

In [15]:
# 이렇게 임베딩까지 해내었다면 훈련을 시켜야 한다.
# 머신러닝 라이브러리 사용 -> 사이킷런
# !pip install -U scikit-learn

   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ------------------------ --------------- 5.0/8.1 MB 26.1 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 31.3 MB/s  0:00:00
   ---------------------------------------- 0.0/37.3 MB ? eta -:--:--
   ------------ --------------------------- 11.3/37.3 MB 54.7 MB/s eta 0:00:01
   ---------------------- ----------------- 21.0/37.3 MB 51.1 MB/s eta 0:00:01
   ---------------------------------- ----- 31.7/37.3 MB 50.8 MB/s eta 0:00:01
   ---------------------------------------- 37.3/37.3 MB 45.4 MB/s  0:00:00

   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   -


[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr = LogisticRegression(max_iter=2000)
# 학습
lr.fit(X_train,y_train)
# 시험
preds = lr.predict(X_test)
# 정확도 확인
print(accuracy_score(y_test,preds))


0.735


In [18]:
msg = "영화 진짜 개 쓰레기네!"

# 긍정 or 부정
# 사용자 메세지 임베딩 처리
msg_embedding = get_embedding([msg],batch_size=1)

# 모델에게 예측시키기
pred_label = lr.predict(msg_embedding)[0]
pred_proba = lr.predict_proba(msg_embedding)[0]
confidence = float(pred_proba[pred_label])

seniment = "긍정" if pred_label == 1 else "부정"
confidence = pred_proba[pred_label]

print(seniment,confidence)

부정 0.5603714541432183


In [24]:
# 비슷한 리뷰 찾기 (코사인 유사도를 이용)

from sklearn.metrics.pairwise import cosine_similarity
msg_embedding = get_embedding([msg],batch_size=1)[0]
similarity = cosine_similarity([msg_embedding],X_train)[0]

# 유사한 리뷰 5개 추출
top_idx = similarity.argsort()[-5:][::-1]

for i in top_idx:
    doc = train_sample.iloc[i]["document"]
    label = train_sample.iloc[i]['label']
    sim = similarity[i]
    print(f"sim = {sim:.3f} {label} | {doc}")


sim = 0.572 0 | ㅋㅋㅋ 진짜 쓰레기네 ㅋ
sim = 0.552 1 | 진짜 대박영화!소름돋을정도로 잘만든영화...
sim = 0.545 0 | OO 낚였네 이거 완전 개빨갱이 영화.. 절대보지마셈 쓰레기임... 시간만 날렸네...
sim = 0.542 0 | 진짜 뭐야 이 영화.. 답없다
sim = 0.540 0 | 전형적인 저질영화
